In [1]:
import sys
sys.path.append('../')
import deepim

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [16]:
deepIm = deepim.DeepIM([100 for i in range(100)],128,[400,400,400],3,0.1)

In [10]:
out = deepIm(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

In [11]:
import onnx
from onnxsim import simplify
import onnxoptimizer
from onnx.shape_inference import infer_shapes

In [12]:
torch.onnx.export(deepIm, torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long), 'deepim.onnx')

In [13]:
onnx_model = onnx.load('deepim.onnx')  # load onnx model
onnx_model = onnxoptimizer.optimize(onnx_model)

onnx_model, check = simplify(onnx_model)
assert check, "Simplified ONNX model could not be validated"
onnx_model = infer_shapes(onnx_model)

onnx.save(onnx_model, 'deepim.onnx')

In [14]:
%timeit -n 1 -r 30 with torch.no_grad(): output = deepIm(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

14.3 ms ± 2.16 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [17]:
interp = utils.ProfilingInterpreter(deepIm)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 107.97119140625 ms
total time: 122.85757064819336 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding              17.6404              14.3584
call_module    mlp_mlp_0                        14.6282              11.9066
call_function  mul                              13.4056              10.9115
call_function  mul_1                            12.8632              10.47
call_function  mul_5                            12.1593               9.89711
call_function  mul_12                           11.4968               9.35781
call_method    sum_3                             1.56784              1.27614
call_method    sum_4                             1.45364              1.18319
call_method    sum_2                             1.39689              1.137
call_method    sum_5                             1.35636              1.10401